In [1]:
import math
import numpy as np

def euler_distance(point1: np.ndarray, point2: list) -> float:
    """
    计算两点之间的欧拉距离，支持多维
    """
    distance = 0.0
    for a, b in zip(point1, point2):
        distance += math.pow(a - b, 2)
    return math.sqrt(distance)

In [2]:
class ClusterNode(object):
    def __init__(self, vec, left=None, right=None, distance=-1, id=None, count=1):
        """
        :param vec: 保存两个数据聚类后形成新的中心
        :param left: 左节点
        :param right:  右节点
        :param distance: 两个节点的距离
        :param id: 用来标记哪些节点是计算过的
        :param count: 这个节点的叶子节点个数
        """
        self.vec = vec
        self.left = left
        self.right = right
        self.distance = distance
        self.id = id
        self.count = count

In [3]:
class Hierarchical(object):
    def __init__(self, k = 1):
        assert k > 0
        self.k = k
        self.labels = None
    def fit(self, x):
        nodes = [ClusterNode(vec=v, id=i) for i,v in enumerate(x)]
        distances = {}
        point_num, future_num = np.shape(x)  # 特征的维度
        self.labels = [ -1 ] * point_num
        currentclustid = -1
        while len(nodes) > self.k:
            min_dist = math.inf
            nodes_len = len(nodes)
            closest_part = None  # 表示最相似的两个聚类
            for i in range(nodes_len - 1):
                for j in range(i + 1, nodes_len):
                    # 为了不重复计算距离，保存在字典内
                    d_key = (nodes[i].id, nodes[j].id)
                    if d_key not in distances:
                        distances[d_key] = euler_distance(nodes[i].vec, nodes[j].vec)
                    d = distances[d_key]
                    if d < min_dist:
                        min_dist = d
                        closest_part = (i, j)
            # 合并两个聚类
            part1, part2 = closest_part
            node1, node2 = nodes[part1], nodes[part2]
            new_vec = [ (node1.vec[i] * node1.count + node2.vec[i] * node2.count ) / (node1.count + node2.count)
                        for i in range(future_num)]
            new_node = ClusterNode(vec=new_vec,
                                   left=node1,
                                   right=node2,
                                   distance=min_dist,
                                   id=currentclustid,
                                   count=node1.count + node2.count)
            currentclustid -= 1
            del nodes[part2], nodes[part1]   # 一定要先del索引较大的
            nodes.append(new_node)
        self.nodes = nodes
        self.calc_label()

    def calc_label(self):
        """
        调取聚类的结果
        """
        for i, node in enumerate(self.nodes):
            # 将节点的所有叶子节点都分类
            self.leaf_traversal(node, i)

    def leaf_traversal(self, node: ClusterNode, label):
        """
        递归遍历叶子节点
        """
        if node.left == None and node.right == None:
            self.labels[node.id] = label
        if node.left:
            self.leaf_traversal(node.left, label)
        if node.right:
            self.leaf_traversal(node.right, label)

In [5]:
from sklearn import datasets
iris = datasets.load_iris()

my = Hierarchical(4)
my.fit(iris.data)
print(np.array(my.labels))

[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 2 1 2 2 2 2 1 2 2 2 2
 2 2 1 1 2 2 2 2 1 2 1 2 1 2 2 1 1 2 2 2 2 2 1 2 2 2 2 1 2 2 2 1 2 2 2 1 2
 2 1]


In [12]:
blognames, words,data = read_file('./data_set/seeds_dataset')

NameError: name 'file' is not defined

In [99]:
from math import sqrt


def read_file(file_name):
    with open(file_name, 'r') as f:
        lines = [line for line in f]
        rownames = []
        data = []
        for line in lines:
            p = line.strip().split('\t')
            rownames.append(p[7])
            #剩余部分就是该行对应的数据
            data.append([ float(x) for x in p[:6]])
    return rownames,data



def pearson(v1,v2):
    #简单求和
    sum1 = sum(v1)
    sum2 = sum(v2)
    
    #求平方和
    sum1_sq = sum([pow(v,2) for v in v1])
    sum2_sq = sum([pow(v,2) for v in v2])
    
    #求乘积之和
    p_sum = sum([v1[i]*v2[i] for i in range(len(v1))])
    
    #计算r
    num = p_sum - (sum1*sum2/len(v1))
    den = sqrt((sum1_sq-pow(sum1,2)/len(v1))*(sum2_sq-pow(sum2,2)/len(v1)))
    if den == 0:
        return 0
    return 1.0-num/den

def h_cluster(rows,distance=pearson):
    distances = {}
    current_cluster_id = -1
    
    #最开始的聚类就是数据集中的行
    cluster = [bi_cluster(rows[i],id=i) for i in range(len(rows))]

    while(len(cluster)>1):
        lowest_pair = (0,1)
        closest = distance(cluster[0].vec,cluster[1].vec)
        
        #遍历每一个配对，寻找最小距离
        for i in range(len(cluster)):
            for j in range(i+1,len(cluster)):
                #用distances来缓存距离的计算值
                if(cluster[i].id,cluster[j].id) not in distances:
                    distances[(cluster[i].id,cluster[j].id)] = distance(cluster[i].vec,cluster[j].vec)
                
                d = distances[(cluster[i].id,cluster[j].id)]
                
                if d<closest:
                    closest=d
                    lowest_pair=(i,j)
        #计算两个聚类的平均值
        merge_vec = [(cluster[lowest_pair[0]].vec[i]+cluster[lowest_pair[1]].vec[i])/2.0 for i in range(len(cluster[0].vec))]
        
        #建立新的聚类
        new_cluster = bi_cluster(merge_vec,left=cluster[lowest_pair[0]],right=cluster[lowest_pair[1]],distance=closest,id=current_cluster_id)
        
        #不在原始集合种的聚类，其id为负数
        current_cluster_id-=1
        del cluster[lowest_pair[1]]
        del cluster[lowest_pair[0]]
        cluster.append(new_cluster)
        
    return cluster[0]

def print_cluster(clust,labels=None,n=0):
    #利用缩进来建立层级布局
    for i in range(n):
        print(' ', end='')
    if clust.id < 0:
        #负数标记代表这是一个分支
        print('-')
    else:
        #正数标记代表这是一个叶节点
        if labels == None:
            print(clust.id)
        else:
            print(labels[clust.id])
    #现在开始打印右侧分支和左侧分支
    if clust.left != None:
        print_cluster(clust.left, labels=labels, n=n+1)
    if clust.right != None:
        print_cluster(clust.right, labels=labels, n= n+1)

class bi_cluster:
    def __init__(self,vec,left=None,right=None,distance=0.0,id=None):
        self.left=left
        self.right=right
        self.vec=vec
        self.id=id
        self.distance=distance
        

In [100]:
def getheight(clusttree):
    """
    绘制树状图--获取聚类数要显示完整需要的高度
    :param clusttree:
    :return:
    """
    #若是叶节点则高度为1
    if clusttree.left == None and clusttree.left == None:
        return 1
    #否则，高度为左右分枝的高度之和
    return getheight(clusttree.left) + getheight(clusttree.right)


def getdepth(clusttree):
    """
    绘制树状图--获取聚类树要显示完整的深度（宽度）
    :param clusttree:
    :return:
    """
    #一个叶节点的距离是0.0
    if clusttree.left == None and clusttree.right == None:
        return 0
    #一个叶节点的距离=左右两侧分支中距离较大者+该支点自身的距离
    return max(getdepth(clusttree.left), getdepth(
        clusttree.right)) + clusttree.distance


def drawnode(draw, clust, x, y, scaling, labels):
    """
    画聚类节点以及子聚类节点
    :param draw:
    :param clust:
    :param x:
    :param y:
    :param scaling:
    :param labels:
    :return:
    """
    if clust.id < 0:
        h1 = getheight(clust.left) * 20
        h2 = getheight(clust.right) * 20
        top = y - (h1 + h2) / 2
        bottom = y + (h1 + h2) / 2
        # 线的长度
        ll = clust.distance * scaling
        # 聚类到其子节点的垂直线
        draw.line((x, top + h1 / 2, x, bottom - h2 / 2), fill=(255, 0, 0))

        # 连接左侧节点的水平线
        draw.line((x, top + h1 / 2, x + ll, top + h1 / 2), fill=(255, 0, 0))

        # 连接右侧节点的水平线
        draw.line((x, bottom - h2 / 2, x + ll, bottom - h2 / 2),
                  fill=(255, 0, 0))

        # 调用函数绘制左右子节点
        drawnode(draw, clust.left, x + ll, top + h1 / 2, scaling, labels)
        drawnode(draw, clust.right, x + ll, bottom - h2 / 2, scaling, labels)
    else:
        # 如果这是一个叶节点，则绘制节点的标签文本
        draw.text((x + 5, y - 7), labels[clust.id], (0, 0, 0))


def drawdendrogram(clusttree, labels, jpeg='clusters.jpg'):
    """
    绘制树状图——为每一个最终生成的聚类创建一个高度为20像素，宽度固定的图片。其中缩放因子是由固定宽度除以总的深度得到的
    :param clusttree:
    :param labels:
    :param jpeg:
    :return:
    """
    # 高度和宽度
    h = getheight(clusttree) * 20
    w = 1200
    depth = getdepth(clusttree)

    # 由于宽度是固定的，因此我们需要对距离值做相应的调整。（因为显示窗口宽度固定，高度可上下拖动）
    scaling = float(w - 150) / depth

    # 新建一个白色背景的图片
    img = Image.new('RGB', (w, h), (255, 255, 255))
    draw = ImageDraw.Draw(img)

    draw.line((0, h / 2, 10, h / 2), fill=(255, 0, 0))

    # 画根节点（会迭代调用画子节点）
    drawnode(draw, clusttree, 10, (h / 2), scaling, labels)
    img.save(jpeg, 'JPEG')


# 数据集转置，进行列聚类。
def rotatematrix(data):
    newdata = []
    for i in range(len(data[0])):
        newrow = [data[j][i] for j in range(len(data))]
        newdata.append(newrow)
    return newdata  #一行表示一个单词在每篇博客中出现的次数

In [101]:
from PIL import Image, ImageDraw
row_names,data = read_file('./data_set/seeds_dataset.txt')
clust = h_cluster(data)  #构建层次聚类树
#print_cluster(clust,labels=row_names)  # 打印聚类树
drawdendrogram(clust, row_names, jpeg='blogclust.jpg')  # 绘制层次聚类树